# Multi-Scenario MORDM

## Imports

In [7]:
# Standard Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import networkx as nx
import scipy as sp

# Workbench Imports
from ema_workbench import (Model, MultiprocessingEvaluator, ScalarOutcome, RealParameter, IntegerParameter, CategoricalParameter, optimize, Scenario, Constant, ema_logging,perform_experiments, SequentialEvaluator)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.util import ema_logging, save_results, load_results
from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator, BaseEvaluator


ema_logging.log_to_stderr(ema_logging.INFO)


<Logger EMA (DEBUG)>

## Model Specification

In [8]:
# Model Imports
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation

dike_model, planning_steps = get_model_for_problem_formulation(6)

In [9]:
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)

## Specify reference scenario
Firstly, we load the results from the open exploration with no action taken to select a reference scenario that shows the most expected number of deaths. This, so-to-say worst-case scenario is then used to find the optimal policies using the MORDM approach.  

In [12]:
experiments, outcomes = load_results("../results/500Scenarios_NoAction_PF1.tar.gz")

[MainProcess/INFO] results loaded succesfully from /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/500Scenarios_NoAction_PF1.tar.gz


In [26]:
outcomes_df = pd.DataFrame(outcomes)

#Get index of worst scenario
index_wc = outcomes_df.sort_values("Expected Number of Deaths").tail(1).index

experiment_wc = experiments.iloc[index_wc]

In [41]:
experiment_wc.iloc[0:,0:19]

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2
347,86.0,145.813371,1.0,0.315934,248.358185,1.5,0.052755,123.774496,1.5,0.057889,81.682672,1.0,0.415533,262.413159,1.5,0.941303,2.5,1.5,3.5


In [29]:
reference_scenario = Scenario(name="Reference", **experiment_wc)
reference_scenario

Scenario({'A.0_ID flood wave shape': 347    86.0
Name: A.0_ID flood wave shape, dtype: float64, 'A.1_Bmax': 347    145.813371
Name: A.1_Bmax, dtype: float64, 'A.1_Brate': 347    1.0
Name: A.1_Brate, dtype: category
Categories (3, float64): [1.0, 1.5, 10.0], 'A.1_pfail': 347    0.315934
Name: A.1_pfail, dtype: float64, 'A.2_Bmax': 347    248.358185
Name: A.2_Bmax, dtype: float64, 'A.2_Brate': 347    1.5
Name: A.2_Brate, dtype: category
Categories (3, float64): [1.0, 1.5, 10.0], 'A.2_pfail': 347    0.052755
Name: A.2_pfail, dtype: float64, 'A.3_Bmax': 347    123.774496
Name: A.3_Bmax, dtype: float64, 'A.3_Brate': 347    1.5
Name: A.3_Brate, dtype: category
Categories (3, float64): [1.0, 1.5, 10.0], 'A.3_pfail': 347    0.057889
Name: A.3_pfail, dtype: float64, 'A.4_Bmax': 347    81.682672
Name: A.4_Bmax, dtype: float64, 'A.4_Brate': 347    1.0
Name: A.4_Brate, dtype: category
Categories (3, float64): [1.0, 1.5, 10.0], 'A.4_pfail': 347    0.415533
Name: A.4_pfail, dtype: float64, 'A.5_Bmax

## Run Optimization on Base Case Scenario

In [30]:
ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(dike_model) as evaluator:
    results_epsilon_0_1 = evaluator.optimize(nfe=1000, epsilons=[0.1,]*len(dike_model.outcomes), reference = reference_scenario)

[MainProcess/INFO] generation 0: 0/1000 nfe
[MainProcess/ERROR] The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Traceback (most recent call last):
  File "/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ema_workbench/em_framework/experiment_runner.py", line 85, in run_experiment
    model.run_model(scenario, policy)
  File "/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ema_workbench/util/ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ema_workbench/em_framework/model.py", line 338, in run_model
    outputs = self.run_experiment(experiment)
  File "/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ema_workbench/util/ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ema_workbench/em_framework/model.py", line 391, in run_experi

EMAError: exception in run_model
Caused by: ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().